<a href="https://colab.research.google.com/github/ShinAsakawa/2019komazawa/blob/master/notebooks/2019komazawa_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title # 2019年度駒澤大学文学部心理学特講IIIA {display-mode: "form"}

name = '氏名'  #@param {type: "string"}
name_kana = 'ふりがな'  #@param {type: "string"}
id = '学生番号'  #@param {type: "string"}
date = '2019-07-16'  #@param {type: "date"}
#@markdown ---


In [0]:
#@title ## 問題 {display-mode: "form"}
#@markdown ### 問1. 深層学習(ディープラーニング)で用いられる以下の略語の本来の綴りを記し，一行程度でその意味を簡潔に説明せよ
term1 = 'CNN: '  #@param {type: "string"}
description1 = '説明:'  #@param {type: "string"}

term2 = 'RNN: '  #@param {type: "string"}
description2 = '説明:'  #@param {type: "string"}

term3 = 'RL: '  #@param {type: "string"}
description3 = '説明: '  #@param {type: "string"}

#@markdown ### 問 2-1. 授業で取り上げた話題の中で <font color='red'>もっとも印象に残っている</font> 話題を一つ挙げよ。また簡単に，1行程度でその理由を記せ
topic2_1 = '話題1: '  #@param {type: "string"}
reason2_1 = '理由1:'  #@param {type: "string"}

#@markdown ### 問 2-2. 授業で取り上げた話題の中で <font color='red'>2番目に印象に残っている</font> 話題を一つ挙げよ。また簡単に，1行程度でその理由を記せ
topic2_2 = '話題2:'  #@param {type: "string"}
reason2_2 = '理由2:'  #@param {type: "string"}

#@markdown ### 問 2-3. 授業で取り上げた話題の中で <font color='red'>もっとも興味が持てなかった</font> 話題を一つ挙げよ。また簡単に，1行程度でその理由を記せ
topic2_3 = '話題3:'  #@param {type: "string"}
reason2_3 = '理由3:'  #@param {type: "string"}
#@markdown ---
x1 = ""



## 空間的注意とディープニューラルネットワークの解釈性

- [オリジナルコード](https://research.google.com/seedbank/seed/spatialchannel_attribution)
- [元論文](https://distill.pub/2018/building-blocks/)

### 準備

In [0]:
!pip install --quiet lucid==0.0.5
!npm install -g svelte-cli@2.2.0

import numpy as np
import tensorflow as tf

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

### モデルの読み込み

In [0]:
model = models.InceptionV1()
model.load_graphdef()

In [0]:
# ラベルの読み込み
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
labels = [line[line.find(" "):].strip() for line in labels_str.split("\n")]
labels = [label[label.find(" "):].strip().replace("_", " ") for label in labels]
labels = ["dummy"] + labels

In [0]:
def raw_class_spatial_attr(img, layer, label, override=None):
  """How much did spatial positions at a given layer effect a output class?"""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    if label is None: return np.zeros(acts.shape[1:-1])
    
    # Compute gradient
    score = T("softmax2_pre_activation")[0, labels.index(label)]
    t_grad = tf.gradients([score], [T(layer)])[0]   
    grad = t_grad.eval({T(layer) : acts})
    
    # Linear approximation of effect of spatial position
    return np.sum(acts * grad, -1)[0]

In [0]:
def raw_spatial_spatial_attr(img, layer1, layer2, override=None):
  """Attribution between spatial positions in two different layers."""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts1 = T(layer1).eval()
    acts2 = T(layer2).eval({T(layer1) : acts1})
    
    # Construct gradient tensor
    # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
    n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
    layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
    score = layer2_mags[n_x, n_y]
    t_grad = tf.gradients([score], [T(layer1)])[0]
    
    # Compute attribution backwards from each positin in layer2
    attrs = []
    for i in range(acts2.shape[1]):
      attrs_ = []
      for j in range(acts2.shape[2]):
        grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
        # linear approximation of imapct
        attr = np.sum(acts1 * grad, -1)[0]
        attrs_.append(attr)
      attrs.append(attrs_)
  return np.asarray(attrs)

In [0]:
def orange_blue(a,b,clip=False):
  if clip:
    a,b = np.maximum(a,0), np.maximum(b,0)
  arr = np.stack([a, (a + b)/2., b], -1)
  arr /= 1e-2 + np.abs(arr).max()/1.5
  arr += 0.3
  return arr

### 空間属性のためのインターフェイス


In [0]:
%%html_define_svelte SpatialWidget

<div class="figure" style="width: 500px; height: 250px; contain: strict;">
  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint1 : spritemap1[pos1[1]][pos1[0]]}}">

    <svg class="pointer_container" viewBox="0 0 {{size1}} {{size1}}">
      {{#each xs1 as x}}
      {{#each ys1 as y}}
        <rect x={{x}} y={{y}} width=1 height=1
          class={{(pos2 != undefined && x == pos2[0] && y == pos2[1])? "selected" : ""}}
          on:mouseover="set({pos2: [x,y], pos1: undefined})"></rect>
      {{/each}}
      {{/each}}
    </svg> 

    <div class="label">{{layer1}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos1: undefined})">
    <img class="img" src="{{img}}">
    <img class="attr" src="{{(pos2 == undefined)? hint2 : spritemap2[pos2[1]][pos2[0]]}}">

    <svg class="pointer_container" viewBox="0 0 {{size2}} {{size2}}">
      {{#each xs2 as x}}
      {{#each ys2 as y}}
        <rect x={{x}} y={{y}} width=1 height=1
          class={{(pos1 != undefined && x == pos1[0] && y == pos1[1])? "selected" : ""}}
          on:mouseover="set({pos1: [x,y], pos2: undefined})"></rect>
      {{/each}}
      {{/each}}
    </svg> 

    <div class="label">{{layer2}}</div>
  </div>
  
</div>


<style>

  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .outer img, .outer svg {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .attr {
    opacity: 0.6;
  }
  .pointer_container {
    z-index: 100;
  }
  .pointer_container rect {
    opacity: 0;
  }
  .pointer_container .selected  {
    opacity: 1;
    fill: none;
    stroke: hsl(24, 100%, 50%);
    stroke-width: 0.1px;
  }
  .label{
    position: absolute;
    left: 0px;
    top: 226px;
    width: 224px;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        hint1: "",
        hint2: "",
        spritemap1 : "",
        size1: 1,
        spritemap2 : "",
        size2: 1,
        pos1: undefined,
        pos2: undefined,
        layer1: "",
        layer2: ""
      };
    },
    computed: {
      xs1: (size1) => range(size1),
      ys1: (size1) => range(size1),
      xs2: (size2) => range(size2),
      ys2: (size2) => range(size2)
    },
    helpers: {range}
  };
</script>

In [0]:
def image_url_grid(grid):
  return [[_image_url(img) for img in line] for line in grid ]

def spatial_spatial_attr(img, layer1, layer2, hint_label_1=None, hint_label_2=None, override=None):
  
  hint1 = orange_blue(
      raw_class_spatial_attr(img, layer1, hint_label_1, override=override),
      raw_class_spatial_attr(img, layer1, hint_label_2, override=override),
      clip=True
  )
  hint2 = orange_blue(
      raw_class_spatial_attr(img, layer2, hint_label_1, override=override),
      raw_class_spatial_attr(img, layer2, hint_label_2, override=override),
      clip=True
  )

  attrs = raw_spatial_spatial_attr(img, layer1, layer2, override=override)
  attrs = attrs / attrs.max()
  
  lucid_svelte.SpatialWidget({
    "spritemap1": image_url_grid(attrs),
    "spritemap2": image_url_grid(attrs.transpose(2,3,0,1)),
    "size1": attrs.shape[3],
    "layer1": layer1,
    "size2": attrs.shape[0],
    "layer2": layer2,
    "img" : _image_url(img),
    "hint1": _image_url(hint1),
    "hint2": _image_url(hint2)
  })

### 簡単な例

In [0]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/dog_cat.png")
spatial_spatial_attr(img, "mixed4a", "mixed4d", hint_label_1="Labrador retriever", hint_label_2="tiger cat")
print "\n両画像の対応関係を調べるためにマウスを画像上で動かしてみてください\n"
print "フニャフニャな耳ととがった耳の違いがラブラドール・レトリーバーとトラネコとの違いであることが分かります"

In [0]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/flowers.png")
spatial_spatial_attr(img, "mixed4a", "mixed4d", hint_label_1="lemon", hint_label_2="vase")
print "\n両画像の対応関係を調べるためにマウスを画像上で動かしてみてください\n"
print "では花瓶とレモンとを分ける特徴は何でしょうか？"

In [0]:
img = load("https://storage.googleapis.com/lucid-static/building-blocks/examples/beer.jpeg")
spatial_spatial_attr(img, "mixed4a", "mixed4d", hint_label_1="beer glass", hint_label_2="wine bottle")
print "\n両画像の対応関係を調べるためにマウスを画像上で動かしてみてください\n"
print "ワインボトルとビール用のグラフを見分けるためにニューロンは何を見ているのでしょうか？"

In [0]:
#https://distill.pub/2018/building-blocks/examples/input_images/dog_cat.jpeg
#https://distill.pub/2018/building-blocks/examples/input_images/chain.jpeg
#https://distill.pub/2018/building-blocks/examples/input_images/beer.jpeg